In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch import tensor
from torch.autograd import Variable
from copy import deepcopy
from random import shuffle
from tensorboardX import SummaryWriter

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
log_path = './output/logs/'
run = int(time.time())
save_dir = output_dir + str(run) + '/'
writer = SummaryWriter()

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [ ]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**6, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**3),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.BCELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = deepcopy(game_board_pt.detach().numpy())
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1.0 if game_board_np[i] > 0.5 else 0.0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size, check_order):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    tile_board = deepcopy(home)
    top_score = org_score
    
    for i in check_order:
        tile_board[i] = 0.0 if home[i] == 1.0 else 1.0
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= top_score:
            loss_board[i] = tile_board[i]
            top_score = tile_result
        else:
            tile_board = deepcopy(loss_board)
    if top_score == org_score:
        shuffle(check_order)
        print('shuffle')
    
    return torch.tensor(loss_board, dtype=torch.float), org_score, top_score, check_order

learning_rate = 0.0002
check_order_p1 = list(range(0, player_size**2))
check_order_p2 = list(range(0, player_size**2))
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

start_p1 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))
start_p2 = Variable(torch.tensor(torch.rand(player_size * player_size), requires_grad=True))

for t in range(10000):
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score, p1_top, check_order_p1 = create_loss_board(p1_board_np, p2_board_np, 300, player_size, check_order_p1)
        p2_loss_board, p2_score, p2_top, check_order_p2 = create_loss_board(p2_board_np, p1_board_np, 300, player_size, check_order_p2)

    loss_p1 = tensor((1 - (p1_score/p1_top))) * loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = tensor((1 - (p2_score/p2_top))) * loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    start_p1 = Variable(torch.tensor(p1_loss_board, requires_grad=True))
    start_p2 = Variable(torch.tensor(p2_loss_board, requires_grad=True))
    
    print(loss_p1, p1_score, p1_top)
    print(loss_p2, p2_score, p2_top)
    
    writer.add_scalars('300/loss', {'loss_p1': loss_p1, 'loss_p2': loss_p2}, t)
    writer.add_scalars('300/top', {'p1_top': p1_top, 'p2_top': p2_top}, t)
    writer.add_scalars('300/score', {'p1_score': p1_score, 'p2_score': p2_score}, t)


    if t % 100 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
        turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
        turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.6184) 21 363
tensor(0.6333) 12 343
Saving players:
tensor([ 0.4500,  0.5322,  0.5407,  0.5351,  0.5056,  0.4839,  0.5101,
         0.5226,  0.4718,  0.4706,  0.5152,  0.5408,  0.4494,  0.4852,
         0.4807,  0.4814,  0.4973,  0.4995,  0.4655,  0.5162,  0.4322,
         0.4918,  0.4599,  0.4971,  0.5349,  0.5294,  0.4307,  0.4574,
         0.5108,  0.4741,  0.5257,  0.4516,  0.4942,  0.4966,  0.4867,
         0.5256,  0.5349,  0.4913,  0.4978,  0.5289,  0.4604,  0.4694,
         0.5402,  0.5309,  0.4803,  0.5067,  0.5100,  0.5092,  0.4653,
         0.5312,  0.4878,  0.4773,  0.5267,  0.5334,  0.5339,  0.5023,
         0.5323,  0.5436,  0.4812,  0.5436,  0.5141,  0.4764,  0.5328,
         0.4847]) tensor([ 0.5221,  0.5289,  0.5064,  0.4914,  0.4689,  0.5384,  0.4876,
         0.4958,  0.4780,  0.5245,  0.4889,  0.4881,  0.4559,  0.5216,
         0.4349,  0.5222,  0.5098,  0.4940,  0.5296,  0.5281,  0.5058,
         0.5280,  0.5377,  0.5260,  0.5162,  0.4896,  0.4930,  0.4775,

tensor(0.5874) 9 169
tensor(0.5168) 54 311
tensor(0.6063) 9 277
tensor(0.5699) 32 378
tensor(0.5984) 9 205
tensor(0.3363) 116 256
tensor(0.3353) 156 332
tensor(0.5103) 54 311
tensor(0.6422) 4 251
tensor(0.4763) 54 246
tensor(0.5892) 10 201
tensor(0.6112) 10 285
tensor(0.6231) 5 325
tensor(0.4884) 54 255
tensor(0.5866) 10 201
tensor(0.5093) 55 293
tensor(0.6206) 3 191
tensor(0.3333) 116 256
tensor(0.5986) 10 183
tensor(0.4724) 54 246
tensor(0.6114) 6 242
tensor(0.3153) 116 238
tensor(0.6131) 10 292
tensor(0.6142) 3 238
tensor(0.5455) 27 209
tensor(0.4774) 54 255
tensor(0.5961) 10 330
tensor(0.4005) 118 328
tensor(0.6323) 4 307
tensor(0.5069) 54 311
tensor(0.5950) 9 179
tensor(0.3154) 125 257
tensor(0.6387) 3 307
tensor(0.5076) 54 311
tensor(0.6037) 5 330
tensor(0.3344) 116 256
tensor(0.5757) 9 149
tensor(0.5643) 32 378
tensor(0.5931) 4 175
tensor(0.5488) 26 246
tensor(0.5904) 4 175
tensor(0.4702) 54 246
tensor(0.5979) 9 213
tensor(0.3891) 116 312
tensor(0.6041) 10 215
tensor(0.5128) 31 

tensor(0.5184) 19 278
tensor(0.3647) 101 390
tensor(0.3405) 0 241
tensor(0.4285) 26 246
tensor(0.2819) 92 327
tensor(0.4399) 56 316
tensor(0.2503) 92 317
tensor(0.3911) 56 316
tensor(0.2499) 92 317
tensor(0.4004) 56 316
tensor(0.3236) 22 300
tensor(0.4629) 26 316
tensor(0.3327) 92 317
tensor(0.4053) 56 316
tensor(0.1554) 156 293
tensor(0.4507) 23 316
tensor(0.1378) 156 293
tensor(0.4746) 23 316
tensor(0.2873) 92 317
tensor(0.3824) 56 316
tensor(0.3479) 16 325
tensor(0.3882) 54 359
tensor(0.3282) 45 259
tensor(0.3927) 54 317
tensor(0.2613) 55 210
tensor(0.3795) 54 301
tensor(0.2889) 27 320
tensor(0.4228) 55 293
tensor(0.2325) 92 317
tensor(0.3909) 56 316
tensor(1.00000e-02 *
       8.3941) 223 327
tensor(0.4296) 54 306
tensor(0.1604) 156 293
tensor(0.4552) 23 316
tensor(0.1525) 156 293
tensor(0.4238) 23 316
tensor(0.1595) 92 241
tensor(0.3611) 56 316
tensor(0.2794) 25 342
tensor(0.4174) 26 316
Saving players:
tensor([ 0.1215,  0.3901,  0.8067,  0.6712,  0.8469,  0.2297,  0.3614,
       

tensor(0.3959) 29 311
tensor(0.1067) 54 294
tensor(0.3437) 18 248
tensor(1.00000e-02 *
       9.6493) 54 294
tensor(0.2359) 92 240
tensor(0.1275) 56 320
tensor(0.3338) 12 361
tensor(0.1104) 54 294
tensor(0.3116) 0 247
tensor(0.1031) 54 294
tensor(0.3675) 0 247
tensor(1.00000e-02 *
       9.5525) 54 294
tensor(0.2103) 156 291
tensor(1.00000e-02 *
       6.1029) 153 269
tensor(0.1957) 92 205
tensor(0.1135) 56 320
tensor(0.2305) 92 240
tensor(0.1008) 56 320
tensor(0.3278) 0 247
tensor(1.00000e-02 *
       9.0209) 54 294
tensor(0.2170) 92 240
tensor(0.1028) 56 320
tensor(0.2983) 0 247
tensor(0.1108) 54 294
tensor(0.2866) 0 247
tensor(1.00000e-02 *
       9.6506) 54 294
tensor(0.3607) 12 361
tensor(1.00000e-02 *
       8.5111) 54 294
tensor(0.2111) 55 241
tensor(0.1525) 26 182
tensor(0.3228) 25 213
tensor(1.00000e-02 *
       9.4121) 54 294
tensor(0.2999) 0 247
tensor(1.00000e-02 *
       9.6532) 54 294
tensor(0.2928) 0 247
tensor(1.00000e-02 *
       7.2261) 54 294
tensor(0.3042) 0 247
ten

tensor(1.00000e-02 *
       8.4487) 105 315
tensor(0.1083) 116 322
tensor(1.00000e-02 *
       8.5586) 105 315
tensor(0.1044) 116 322
tensor(0.1602) 105 315
tensor(0.1016) 116 322
tensor(1.00000e-02 *
       9.3469) 105 315
tensor(0.1108) 116 322
tensor(0.1249) 133 247
tensor(0.4316) 17 237
tensor(1.00000e-02 *
       5.4098) 215 315
tensor(0.1188) 155 313
tensor(1.00000e-02 *
       7.2390) 133 192
tensor(0.3748) 17 237
tensor(1.00000e-02 *
       5.2720) 215 315
tensor(0.1084) 155 313
tensor(0.1476) 3 315
tensor(1.00000e-02 *
       8.9776) 116 224
tensor(0.1141) 105 315
tensor(0.1054) 116 322
tensor(1.00000e-02 *
       4.7329) 215 315
tensor(0.1078) 155 313
tensor(0.1317) 3 315
tensor(1.00000e-02 *
       4.3173) 116 224
tensor(1.00000e-02 *
       9.4155) 105 315
tensor(1.00000e-02 *
       9.7787) 116 322
tensor(1.00000e-02 *
       3.3046) 215 315
tensor(1.00000e-02 *
       9.1270) 155 313
tensor(1.00000e-02 *
       8.6206) 105 315
tensor(0.1530) 116 322
tensor(1.00000e-02 *
 

...

tensor(1.00000e-02 *
       9.3300) 114 239
tensor(1.00000e-04 *
       2.8530) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       6.6101) 164 201
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       5.0730) 164 201
shuffle
shuffle
tensor(0.) 233 233
tensor(0.) 264 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       2.7007) 164 264
tensor(0.1680) 86 215
tensor(1.00000e-02 *
       5.1575) 139 263
shuffle
tensor(0.) 354 354
tensor(0.1346) 147 240
shuffle
tensor(0.) 354 354
tensor(0.1288) 147 240
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       2.8372) 164 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       3.0177) 164 264
shuffle
shuffle
tensor(0.) 233 233
tensor(0.) 264 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       8.1783) 164 269
tensor(1.00000e-02 *
       8.8466) 114 239
tensor(1.00000e-04 *
       2.2433) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       6.3457) 164 269
tensor(1.00000e-02 *
       6.6048) 114 188
tensor(1.

shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       6.7602) 164 269
tensor(0.1254) 114 277
tensor(1.00000e-04 *
       3.4495) 263 264
tensor(0.1234) 114 277
tensor(1.00000e-04 *
       3.1369) 263 264
tensor(0.1265) 114 277
tensor(1.00000e-04 *
       2.8498) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       6.3464) 164 269
tensor(0.1797) 114 277
tensor(1.00000e-04 *
       4.6504) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       5.7392) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       5.9069) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       8.2025) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.9684) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       5.8307) 164 269
tensor(0.1016) 114 277
tensor(1.00000e-04 *
       3.6102) 263 264
tensor(0.1052) 114 277
tensor(1.00000e-04 *
       3.0151) 263 264
tensor(0.2124) 86 240
tensor(1.00000e-02 *
       6.1515) 139 272
shuffle
tensor(0.) 263 263
tensor(1

tensor(0.1240) 114 277
tensor(1.00000e-03 *
       1.0422) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.3980) 164 269
tensor(0.2417) 44 273
tensor(1.00000e-02 *
       6.5999) 203 443
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.8403) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.3091) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.2194) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.4157) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.5453) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       5.4173) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.8193) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       6.3359) 164 269
tensor(1.00000e-02 *
       9.5620) 114 239
tensor(1.00000e-04 *
       5.8386) 263 264
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       4.2810) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       3.2936) 164 26

shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       2.9617) 164 269
tensor(0.1697) 155 289
tensor(1.00000e-02 *
       2.3270) 186 269
tensor(0.2621) 87 308
tensor(1.00000e-02 *
       4.3183) 110 265
tensor(0.1562) 103 261
tensor(1.00000e-02 *
       1.9275) 202 265
tensor(0.1398) 103 261
tensor(1.00000e-02 *
       1.4394) 202 265
tensor(0.2668) 87 308
tensor(1.00000e-02 *
       5.8333) 110 265
tensor(0.1017) 114 277
tensor(1.00000e-04 *
       3.2835) 263 264
tensor(0.3411) 90 307
tensor(0.3471) 0 223
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       2.9418) 164 269
tensor(0.1079) 103 386
tensor(1.00000e-02 *
       1.4133) 202 265
tensor(0.3670) 90 307
tensor(0.2996) 0 223
tensor(1.00000e-02 *
       7.0458) 155 414
tensor(1.00000e-02 *
       2.8179) 186 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       3.4060) 164 269
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       3.4080) 164 269
tensor(0.2302) 103 344
tensor(1.00000e-02 *
       1.4501) 202 265
tensor(0

tensor(1.00000e-02 *
       9.4220) 155 414
tensor(1.00000e-02 *
       7.8992) 186 332
tensor(0.2344) 103 315
tensor(1.00000e-02 *
       7.5285) 202 355
tensor(0.1570) 103 333
tensor(1.00000e-02 *
       7.4557) 202 355
tensor(1.00000e-02 *
       7.3790) 155 414
tensor(1.00000e-02 *
       7.2883) 186 332
tensor(0.2082) 103 315
tensor(1.00000e-02 *
       6.9285) 202 355
tensor(1.00000e-02 *
       8.2898) 155 414
tensor(1.00000e-02 *
       7.9922) 186 332
tensor(0.1528) 103 333
tensor(1.00000e-02 *
       6.8067) 202 355
tensor(0.1527) 87 284
tensor(0.1168) 110 237
tensor(0.1746) 103 315
tensor(1.00000e-02 *
       6.8961) 202 355
shuffle
tensor(0.) 263 263
tensor(1.00000e-02 *
       8.7205) 164 332
tensor(0.1842) 103 275
tensor(1.00000e-02 *
       7.2842) 202 355
tensor(0.2387) 87 305
tensor(0.1245) 110 237
tensor(0.1740) 103 275
tensor(1.00000e-02 *
       6.4222) 202 355
tensor(0.1979) 103 275
tensor(1.00000e-02 *
       6.4228) 202 355
tensor(0.1802) 103 275
tensor(1.00000e-